# Read Data

In [1]:
from re import template
import pandas as pd
from utils.algo.calculation import get_fbna
from dash_bootstrap_templates import load_figure_template
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# import talib
import pandas as pd

templates = ["solar"]
load_figure_template(templates)

In [2]:
df = pd.read_csv("./data/2317TW.csv")

In [3]:
templates = ["solar"]
load_figure_template(templates)

In [32]:
fig = go.FigureWidget(go.Figure())
fig.add_trace(go.Candlestick(x=df["Date"],
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name="Price", 
                            ))
fig.update_layout(
    yaxis = dict(
       autorange = True,
       fixedrange= False
   ), width=1000*0.7,height=700*0.8,template="solar"
)

FigureWidget({
    'data': [{'close': array([ 61.5868454 ,  59.21202087,  57.94546127, ..., 110.        ,
    …

In [35]:
scatter = fig.data[0]
def log_date(trace, points, selector):
    global fuck
    date = list(scatter.x)
    for i in points.point_inds:
        fuck.append(date[i])

scatter.on_click(log_date)
fig

FigureWidget({
    'data': [{'close': array([ 61.5868454 ,  59.21202087,  57.94546127, ..., 110.        ,
    …

In [75]:
fuck = []

In [66]:
fuck

['2015-01-13', '2015-09-02', '2016-01-29', '2015-08-25']

In [59]:
import time
import threading

In [76]:
def clear_timeli():
    while True:
        if len(fuck)>2:
            fuck = []

In [77]:
t = threading.Thread(target = clear_timeli)

In [ ]:
t.start()

Exception in thread Thread-5 (clear_timeli):
Traceback (most recent call last):
  File "c:\Users\zhing\anaconda3\envs\tmp\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Users\zhing\anaconda3\envs\tmp\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\zhing\AppData\Local\Temp\ipykernel_21268\1503091534.py", line 3, in clear_timeli


UnboundLocalError: local variable 'fuck' referenced before assignment
